## Code generator

In [1]:
import os 
from dotenv import load_dotenv
from openai import OpenAI
import subprocess
from IPython.display import display, Markdown

In [2]:
!ollama list

NAME                       ID              SIZE      MODIFIED     
qwen2.5-coder:latest       dae161e27b0e    4.7 GB    2 weeks ago     
gpt-oss:latest             17052f91a42e    13 GB     3 weeks ago     
llama3.2:3b                a80c4f17acd5    2.0 GB    4 weeks ago     
llama3.1:8b                46e0c10c039e    4.9 GB    3 months ago    
nomic-embed-text:latest    0a109f422b47    274 MB    5 months ago    


In [3]:
MODEL = "gpt-oss:latest"

OLLAMA_BASE_URL = "http://localhost:11434/v1"
ollama = OpenAI(base_url=OLLAMA_BASE_URL, api_key="ollama")

In [4]:
from system_info import retrieve_system_info

system_info = retrieve_system_info()
system_info

{'os': {'system': 'Darwin',
  'arch': 'arm64',
  'release': '24.6.0',
  'version': 'Darwin Kernel Version 24.6.0: Mon Aug 11 21:15:09 PDT 2025; root:xnu-11417.140.69.701.11~1/RELEASE_ARM64_T6041',
  'kernel': '24.6.0',
  'distro': None,
  'wsl': False,
  'rosetta2_translated': False,
  'target_triple': 'arm64-apple-darwin24.6.0'},
 'package_managers': ['xcode-select (CLT)', 'brew'],
 'cpu': {'brand': 'Apple M4 Max',
  'cores_logical': 14,
  'cores_physical': 14,
  'simd': []},
 'toolchain': {'compilers': {'gcc': 'Apple clang version 17.0.0 (clang-1700.3.19.1)',
   'g++': 'Apple clang version 17.0.0 (clang-1700.3.19.1)',
   'clang': 'Apple clang version 17.0.0 (clang-1700.3.19.1)',
   'msvc_cl': ''},
  'build_tools': {'cmake': '', 'ninja': '', 'make': 'GNU Make 3.81'},
  'linkers': {'ld_lld': ''}}}

In [7]:
message = f"""
Here is a report of the system information for my computer.
I want to run a C++ compiler to compile a single C++ file called main.cpp and then execute it in the simplest way possible.
Please reply with whether I need to install any C++ compiler to do this. If so, please provide the simplest step by step instructions to do so.

If I'm already set up to compile C++ code, then I'd like to run something like this in Python to compile and execute the code:
```python
compile_command = # something here - to achieve the fastest possible runtime performance
compile_result = subprocess.run(compile_command, check=True, text=True, capture_output=True)
run_command = # something here
run_result = subprocess.run(run_command, check=True, text=True, capture_output=True)
return run_result.stdout
```
Please tell me exactly what I should use for the compile_command and run_command.

System information:
{system_info}
"""

response = ollama.chat.completions.create(
    model = MODEL, 
    messages=[{"role": "user", "content": message}],
)

display(Markdown(response.choices[0].message.content))

### Short answer  
No. Your machine already ships with a fully‑featured C++ compiler (`clang++` / `g++` – both are the same “Apple clang 17” that came with Xcode CLT).  
You can compile and run a single file with no extra installation.

---

## 1. Quick sanity check (optional)

```sh
$ clang++ --version
Apple clang version 17.0.0 (clang-1700.3.19.1)
Target: arm64-apple-darwin24.6.0
Thread model: posix
$ which clang++
/Library/Developer/CommandLineTools/usr/bin/clang++
```

If, by any chance, you get “command not found”, run the one‑time command

```sh
xcode-select --install    # installs the Command Line Tools
```

but in your case this is already present.

---

## 2. Fastest‑possible compilation flags for a single‑file program

| Flag | Purpose |
|------|---------|
| `-O3` | Full optimisation (including inlining, loop‑unrolling, etc.) |
| `-std=c++20` (or `c++17` / `c++23` if you prefer) | Use the latest language standard you need |
| `-march=native` (optional) | Enable architecture‑specific optimisations (most compilers enable this by default on macOS). |
| `-ffast-math` (optional) | Allow aggressive IEEE‑754 optimisations (only if you are sure it won’t break correctness). |
| `-pipe` (optional) | Reduce disk I/O during compilation (default on macOS). |

In practice the simplest, safest, and still fully‑optimised invocation is:

```sh
clang++ -O3 -std=c++20 main.cpp -o main
```

---

## 3. Python `subprocess` snippet

You can use the `subprocess.run` interface exactly as shown in your question.  
Below are the two lines you should replace the comments with:

```python
import subprocess

# ---- Compile ---------------------------------------------------
compile_command = [
    "clang++",          # compiler
    "-O3",              # optimisation level (fastest runtime)
    "-std=c++20",       # language standard (change if you want C++17/C++23)
    "main.cpp",         # source file
    "-o", "main"        # output executable name
]

compile_result = subprocess.run(
    compile_command,
    check=True,
    text=True,
    capture_output=True
)

# ---- Run -------------------------------------------------------
run_command = ["./main"]          # relative invocation of the binary

run_result = subprocess.run(
    run_command,
    check=True,
    text=True,
    capture_output=True
)

print(run_result.stdout)          # <-- output of the program
```

**Why this works**

* `compile_command` is a list – this is the safest form for `subprocess.run` because it bypasses the shell and handles quoting for you.
* `run_command` is also a list; `./main` is the typical Unix way to execute a binary in the current directory.

---

## 4. Optional shortcuts

* **Make a single‑command wrapper** (quick and handy from the shell):

  ```sh
  # Compile
  clang++ -O3 -std=c++20 main.cpp -o main

  # Run
  ./main
  ```

* **Combine on the fly** (one line, no intermediate binary saved):

  ```sh
  clang++ -O3 -std=c++20 main.cpp -o /tmp/main && /tmp/main
  ```

* **Using `g++` instead of `clang++`** – they are identical on macOS, just swap the command name if you prefer:

  ```python
  compile_command = ["g++", "-O3", "-std=c++20", "main.cpp", "-o", "main"]
  ```

---

## 5. Bottom line

* **No extra compiler installation needed** – you can compile and run immediately.  
* Use the `clang++` command shown above (or the `g++` alias) with `-O3 -std=c++20`.  
* In Python, `compile_command` is the list of compiler arguments, `run_command` is simply `["./main"]`.  

Good luck with your C++ coding on the Apple M4 Max!

In [8]:
compile_command = ["clang++", "-O3", "-std=c++20", "main.cpp" ,"-o", "main"]
run_command = ["./main"]

In [9]:
system_prompt = """
Your task is to convert Python code into high performance C++ code.
Respond only with C++ code. Do not provide any explanation other than occasional comments.
The C++ response needs to produce an identical output in the fastest possible time.
"""

def user_prompt_for(python):
    return f"""
Port this Python code to C++ with the fastest possible implementation that produces identical output in the least time.
The system information is:
{system_info}
Your response will be written to a file called main.cpp and then compiled and executed; the compilation command is:
{compile_command}
Respond only with C++ code.
Python code to port:

```python
{python}
```
"""

In [10]:
def messages_for(python):
    return [
        {"role": "system", "content": system_prompt},
        {"role": "user", "content": user_prompt_for(python)}
    ]
 

In [11]:
def write_output(cpp):
    with open("main.cpp", "w", encoding="utf-8") as f:
        f.write(cpp)

In [12]:
def port(model, python):
    response = ollama.chat.completions.create(
        model = model,
        messages=messages_for(python),
        reasoning_effort="high"
    )

    reply = response.choices[0].message.content
    reply = reply.replace('```cpp','').replace('```','')
    write_output(reply)

In [13]:
pi = """
import time

def calculate(iterations, param1, param2):
    result = 1.0
    for i in range(1, iterations+1):
        j = i * param1 - param2
        result -= (1/j)
        j = i * param1 + param2
        result += (1/j)
    return result

start_time = time.time()
result = calculate(200_000_000, 4, 1) * 4
end_time = time.time()

print(f"Result: {result:.12f}")
print(f"Execution Time: {(end_time - start_time):.6f} seconds")
"""

In [14]:
def run_python(code):
    globals = {"__builtins__": __builtins__}
    exec(code, globals)

In [15]:
run_python(pi)

Result: 3.141592656089
Execution Time: 10.466765 seconds


In [16]:
port(MODEL, pi)

In [17]:
# Use the commands from gpt-oss-latest

def compile_and_run():
    subprocess.run(compile_command, check=True, text=True, capture_output=True)
    print(subprocess.run(run_command, check=True, text=True, capture_output=True).stdout)
    print(subprocess.run(run_command, check=True, text=True, capture_output=True).stdout)
    print(subprocess.run(run_command, check=True, text=True, capture_output=True).stdout)

In [18]:
compile_and_run()

Result: 3.141592656089
Execution Time: 0.235518 seconds

Result: 3.141592656089
Execution Time: 0.215391 seconds

Result: 3.141592656089
Execution Time: 0.218132 seconds



Time taken by python code = 10.466765 seconds           
Avg time taken by gpt-oss generated cpp code = 0.223017 seconds 